In [1]:
from models.rnn.rnn_plain import RNN_Plain
from models.rnn.rnn_attention import RNN_Attention

from lang import load_data_int_seq
from utils import accuracy_score
%load_ext autoreload
%autoreload 2

C:\Users\danie\anaconda3\envs\drlnd\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
output_lang, input_lang, train, X_test, y_test = load_data_int_seq()

## Plain RNN

In [3]:
algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 2, input_size=input_lang.n_words, hidden_size=256, output_size=output_lang.n_words, embedding_size = 28, calc_magnitude_on=False)

In [22]:
algo.train(input_lang, output_lang, train)

original pairs
[('24,28,32,36,40,44,48,52', '4*t+4+2*8'), ('36,37,38,39,40,41,42,43', '0+5*8-5+t'), ('10,11,12,13,14,15,16,17', '1+0+t-1+9')]
input data
['-8,-21,-34,-47,-60,-73,-86,-99', '-7,-6,-5,-4,-3,-2,-1,0', '36,37,38,39,40,41,42,43']
target data
['t+5-t*7*2', 't-2-6+3*0', '0+5*8-5+t']
=====> minibatches:
[tensor([[40, 47, 48, 49, 50, 51, 52, 53,  0],
        [24, 25, 26, 27, 28, 29, 30, 31,  0],
        [ 5, 10, 11, 12,  6, 13, 14, 15,  0]]), tensor([[43, 41, 33, 42, 16, 18, 20, 22,  0],
        [37, 38, 39, 40, 27, 31, 41, 42,  0],
        [ 2,  3,  4,  5,  6,  7,  8,  9,  0]])]
------
[tensor([[ 4,  5,  9, 10,  4,  3, 15,  3,  6,  0],
        [ 4, 10,  6, 10, 13,  5, 14,  3,  8,  0],
        [ 8,  5,  9,  3,  7, 10,  9,  5,  4,  0]]), tensor([[ 8,  5,  4, 10,  4,  5,  4,  3,  6,  0],
        [13,  5,  2,  3,  4, 10,  9,  3, 13,  0],
        [ 2,  3,  4,  5,  2,  5,  6,  3,  7,  0]])]
<===== minibatches:
0m 0s (- 0m 0s) (1 50%) 2.6475
0m 0s (- 0m 0s) (2 100%) 2.6867


In [ ]:
pred = algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

In [8]:
print(',' in '-7,-6,-5,-4,-3,-2,-1,0')

True


In [9]:
print('-7,-6,-5,-4,-3,-2,-1,0' is not '')

True


### Plain RNN with Magnitude loss

In [ ]:
magn_algo = RNN_Plain(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
magn_algo.train(input_lang, output_lang, train)

In [ ]:
pred = magn_algo.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

##  Attention-based

In [ ]:
algo_attn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=False)
algo_attn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])

### Attention with magnitude loss

In [ ]:
algo_attn_magn = RNN_Attention(symbols = "+*-0123456789t", output_sequence_length = 9, encoded_seq_length = 9, num_epochs= 5000, input_size=input_lang.n_words, hidden_size=512, output_size=output_lang.n_words, calc_magnitude_on=True)
algo_attn_magn.train(input_lang, output_lang, train)

In [ ]:
pred = algo_attn_magn.infer(input_lang, output_lang, X_test[:1000])
pred[:25]

In [ ]:
accuracy_score(pred, y_test[:1000])